<a href="https://colab.research.google.com/github/hikmat690/AI-programming/blob/main/bbcnewspaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis (Text Classification)**
*   **Downloading Datset from Kaggle to Google Colab**
*   **Text Cleaning**
*   **BERT Model (Feature Engineering)**
*   **DL Model**

In [49]:
!pip install -U "tensorflow-text==2.13.*"

# **Importing Preprocessing Libraries**

In [50]:
#!pip install --quiet tensorflow_text

import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,precision_score,accuracy_score,confusion_matrix

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# **Reading Data**

In [51]:
temp_df = pd.read_csv('/content/bbc-text.csv')
df = temp_df.iloc[:1000]

In [52]:
df.shape

(1000, 2)

# **Text Cleaning & Preprocessing**

In [53]:

def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

#exclude = "!.,?"
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords])

In [54]:
df['text'] = df['text'].str.lower()

df['text'] = df['text'].apply(remove_html_tags)

df['text'] = df['text'].apply(remove_url)

df['text'] = df['text'].apply(remove_punc)
df['text'] = df['text'].apply(remove_stopwords)

<ipython-input-54-6b16f0a4fee3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
<ipython-input-54-6b16f0a4fee3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(remove_html_tags)
<ipython-input-54-6b16f0a4fee3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [55]:
df.head()

,category,text
0,tech,tv future hands viewers home theatre systems p...
1,business,worldcom boss left books alone former worldcom...
2,sport,tigers wary farrell gamble leicester say rushe...
3,sport,yeading face newcastle fa cup premiership side...
4,entertainment,ocean twelve raids box office ocean twelve cri...


# **Feature Engineering**

**Target Column Encoding**

In [56]:

from sklearn.preprocessing import LabelEncoder

X = df['text']
Y = df['category']
#print(X)
#print(Y)

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

#print(Y)

X_train,X_test,y_train,y_test = train_test_split(df['text'],Y,test_size=0.2,random_state=42)
print(X_train)
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

29     halloween writer debra hill dies screenwriter ...
535    pc ownership double 2010 number personal compu...
695    isinbayeva claims new world best pole vaulter ...
557    us bank boss hails genius smith us federal res...
836    tsunami cost hits jakarta shares stock market ...
                             ...                        
106    mcclaren eyes uefa cup top spot steve mcclaren...
270    building giant asbestos payout australian buil...
860    new year texting breaks record mobile phone es...
435    mac mini heralds mini revolution mac mini laun...
102    targets better many economic targets set lisbo...
Name: text, Length: 800, dtype: object
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


**Finetuning using Deep Learning**

In [57]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",trainable=True)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
#print (text_input)
encoder_inputs = preprocessor(text_input)
#print(encoder_inputs)
outputs = encoder(encoder_inputs)
#print(outputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
#print(pooled_output)

drop_out = tf.keras.layers.Dropout(0.2,name='dropout')(pooled_output)
output = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(drop_out)

model=tf.keras.Model(inputs=[text_input],outputs=[output])

In [1]:
# Compile the mode
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, validation_split=0.1)

NameError: name 'model' is not defined

In [ ]:
def get_bert_embeddings(text_data):
    # Convert the pandas Series to a list of strings
    text_data = text_data.tolist()
    # Now pass the list of strings to the preprocessor
    preprocessed_text = preprocessor(text_data)
    outputs = encoder(preprocessed_text)
    pooled_output = outputs["pooled_output"]
    return pooled_output

train_embeddings = get_bert_embeddings(X_train)
test_embeddings = get_bert_embeddings(X_test)

print(train_embeddings.shape)
print(type(train_embeddings))

In [ ]:
# Initialize and train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(train_embeddings, y_train)

# Predict on the test set
y_pred = rf_model.predict(test_embeddings)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(train_embeddings, y_train)
rf_model.fit(train_embeddings, y_train)

# Predict on the test set
y_pred = rf_model.predict(test_embeddings)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))
